# Predicting Education Level

## Introduction

At some point in our lives we must decide on how to make a living. People wish to find a path that leads to the most success without using too many resources, and one of the main problems to that is education. The decision to pursue a higher education is both time and money-consuming, which is why our Data Science project question is: "Can we predict the highest level of education someone has received based on their salary, years of experience, and age?"

The dataset we are using contains 7 columns reading left to right: `Education`, `Experience`, `Location`, `Job_Title`, `Age`, `Gender`, `Salary`, where each row represents an individual worker.

- `Education`: four categories representing the highest level of education received: High School, Bachelor, Master, PhD
- `Experience`: years of professional experience
- `Location`: three locations of work: Urban, Suburban, Rural
- `Job_Title`: four general job titles: Manager, Director, Analyst, Engineer
- `Age`: age
- `Gender`: Female or Male
- `Salary`: annual salary

## Preliminary exploratory data analysis

- Demonstrate that the dataset can be read from the web into R

We found the dataset on Kaggle. We were able to download it into the project folder as a `.csv` file. After that, we are able to use the `read_csv` function to read it into a data frame to then work with in `R`. The name of the file is "`salary_prediction_data.csv`" and it's stored in the `data` folder.

In [1]:
# Load in the libraries needed to read the csv file
library(tidyverse)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Load the csv file into a data frame
df_untidy <- read_csv("data/salary_prediction_data.csv")
head(df_untidy)

Rows: 1000 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Education, Location, Job_Title, Gender
dbl (3): Experience, Age, Salary

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Education,Experience,Location,Job_Title,Age,Gender,Salary
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>
High School,8,Urban,Manager,63,Male,84620.05
PhD,11,Suburban,Director,59,Male,142591.26
Bachelor,28,Suburban,Manager,61,Female,97800.26
High School,29,Rural,Director,45,Male,96834.67
PhD,25,Urban,Analyst,26,Female,132157.79
PhD,19,Rural,Director,27,Female,156312.94


- Clean and wrangle your data into a tidy format

Our dataset is already close to being tidy, but there are a few improvements we could do to make it better. We could change all the predictor names to be lowercase and include the units that it is in. Looking at the salary column (in our original `csv` file), we can see the values are all not rounded to 2 decimal places. We also need to remove the Location, Job_Title, and Gender columns because those are not going to be used as predictors in our analysis.

In [3]:
# Rename all the columns to include the units and be in lowercase
# Then select the newly mutated columns only
df_tidy <- df_untidy |>
    mutate(education = Education, experience_years = Experience, age_years = Age, salary_dollars = Salary) |>
    select(education, experience_years, age_years, salary_dollars) |>
    mutate(salary_dollars = round(salary_dollars, 2))

head(df_tidy)

education,experience_years,age_years,salary_dollars
<chr>,<dbl>,<dbl>,<dbl>
High School,8,63,84620.05
PhD,11,59,142591.26
Bachelor,28,61,97800.26
High School,29,45,96834.67
PhD,25,26,132157.79
PhD,19,27,156312.94


- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data.

In [4]:
# Load library for spliting data into training and testing
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Search for functions across packages at https://www.tidymodels.org/find/



In [5]:
# Split data into a rough 75-25 split
df_split <- initial_split(df_tidy, 0.75, education)
df_train <- training(df_split)

# Summarise the average values for each of our predictor classes
df_summary_1 <- df_train |>
    filter(row_number() <= training_amount) |>
    summarise(average_experience_years = round(mean(experience_years, na.rm=TRUE)),
              average_age_years = round(mean(age_years, na.rm=TRUE)),
              average_salary_dollars = round(mean(salary_dollars, na.rm=TRUE), 2))

# Summarise the total observations in each predictor class
df_summary_2 <- df_train |>
    filter(row_number() <= training_amount) |>
    summarise(total_observations = length(experience_years))

# There are no observations with missing data.

# Print them out
print(df_split)
head(df_summary_1)
head(df_summary_2)

ERROR: [1m[33mError[39m in `filter()`:[22m
[1m[22m[36mℹ[39m In argument: `row_number() <= training_amount`.
[1mCaused by error:[22m
[33m![39m object 'training_amount' not found


- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [ ]:
df_plot <- df_train |>
    ggplot(aes(x = age_years, y = salary_dollars, color = education)) +
    geom_point() +
    labs(x = "Age (years)", y = "Salary ($)", color = "Education Level")

df_plot

## Methods

To help answer our question, we plan to use the age, experience, and salary columns within the data set. Instead of gender and location, which lack any significant relationship to a person's education, these can be applied to many people. Furthermore, while the job title variable may have some correlations, there is too much variance in the levels of education for each title that it is not suitable. Suppose a high school student looking into jobs at a certain salary range, then this data can provide insight into the average age, experience, and education standard for the desired amount. Additionally, by using graphs to visualize this information, people can make an informed decision and thus plan out their future accordingly.

## Expected outcomes and significance

We expect to find that higher numbers in all predictor fields (salary, years of work experience, and age) predict a higher level of education.

We wish to use our findings to help someone determine what education level, years of experience, and age they should be in order to reach a certain salary range. These results may impact the way people choose to allocate the years of their life, and hopefully help them reach a decision in a less ambiguous way.

This analysis could lead to future questions about whether or not people should be spending their time and money on higher education in order to reach their desired financial position.